In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker



In [3]:
years = list(range(2001, 2022))
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=FreeThrowsAttempted&agg=PerGame"

In [12]:
for year in years:
    with open("attempted_freethrows_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [13]:
dfs = []
for year in years:
    with open("attempted_freethrows_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    freethrows_data = pd.read_html(str(player_table))[0]
    freethrows_data["Year"] = year
    dfs.append(freethrows_data)

In [14]:
freethrows_data = pd.concat(dfs)
freethrows_data

,Rank,Player,Team,GamesG,AttemptedAtt.,Made,% Made%,Year
0,1,"SKELIN, MATESKELIN, M.",MESKRKA Novo MestoKRKA Novo Mesto,14,8.57,6.00,70.00%,2001
1,2,"BODIROGA, DEJANBODIROGA, D.",PAOPanathinaikos AthensPanathinaikos Athens,22,7.36,5.86,79.63%,2001
2,3,"LAKOVIC, JAKALAKOVIC, J.",MESKRKA Novo MestoKRKA Novo Mesto,14,7.14,6.36,89.00%,2001
3,4,"FUCKA, GREGORFUCKA, G.",FORSkipper BolognaSkipper Bologna,20,6.95,4.45,64.03%,2001
4,5,"FORD, ALPHONSOFORD, A.",OLYOlympiacosOlympiacos,20,6.75,5.35,79.26%,2001
...,...,...,...,...,...,...,...,...
45,46,"POIRIER, VINCENTPOIRIER, V.",RMBReal MadridReal Madrid,37,2.22,1.70,76.83%,2021
46,47,"HINES, KYLEHINES, K.",AXMAX Armani Exchange MilanAX Armani Exchange ...,36,2.19,1.44,65.82%,2021
47,48,"NEBO, JOSHNEBO, J.",ZALZalgiris KaunasZalgiris Kaunas,28,2.18,1.46,67.21%,2021
48,49,"WHITE, OKAROWHITE, O.",PAOPanathinaikos OPAP AthensPanathinaikos OPAP...,29,2.17,1.79,82.54%,2021


In [15]:
engine = create_engine('sqlite:///freethrows_attempted_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [16]:
freethrows_data.to_sql(
    name = "freethrows_attempted_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [ ]:
freethrows_attempted_leaders_select = session.execute('select * from "freethrows_attempted_leaders"')
freethrows_attempted_leaders_select.all()